# Spotify Exploration

As of now (8/17/2023) I have only done this in R. So before we create the functions, we will recreate the functionality of my R script. 

1. Install Necessary Packages
2. Set system environments appropriately for Spotify ID's
3. Experiment with the Spotipy package
a. Get songs from an artist
b. Using track_id, get the song info (length, tempo, danceability, energy)
4. How many songs do I have saved?
5. Read in names of all songs
6. Get relevant data from all songs
7. Manage the dataframe
8. Create playlist groupings by tempo
9. Create playlist groupings by danceability
10. Create playlist groupings by energy
11. Write Playlist
12. Work with functions

# 1. Import Packages

In [1]:
import config
import pandas as pd
import numpy as np
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# 2. Authorization

Here we'll keep an example code from Spotipy for client authorizaiton (search spotify database but not any user data)
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

playlists = sp.user_playlists('spotify')
while playlists:
    for i, playlist in enumerate(playlists['items']):
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

Set system environments

In [5]:
os.environ["SPOTIPY_CLIENT_ID"] = config.SPOTIPY_CLIENT_ID
os.environ["SPOTIPY_CLIENT_SECRET"] = config.SPOTIPY_CLIENT_SECRET
os.environ["SPOTIPY_REDIRECT_URI"] = config.SPOTIPY_REDIRECT_URI

Set scope & do a data pull

* Note that "user-library-read" is the only necessary scope to read saved tracks

In [6]:
scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

profile = sp.current_user()
results = sp.current_user_saved_tracks(limit = 20, offset = 0, market = None)

# Item is a specific "row". Or a single song
for idx, item in enumerate(results['items']):
    
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])

0 Lil Uzi Vert  –  Watch This - ARIZONATEARS Pluggnb Remix
1 Post Malone  –  Don't Understand
2 Dominic Fike  –  3 Nights
3 Dominic Fike  –  Double Negative (Skeleton Milkshake)
4 Post Malone  –  Something Real
5 Lit  –  My Own Worst Enemy
6 J. Cole  –  h u n g e r . o n . h i l l s i d e (with Bas)
7 J. Cole  –  a p p l y i n g . p r e s s u r e
8 J. Cole  –  m y . l i f e (with 21 Savage & Morray)
9 J. Cole  –  p u n c h i n ‘ . t h e . c l o c k
10 J. Cole  –  9 5 . s o u t h
11 J. Cole  –  a m a r i
12 BlocBoy JB  –  Look Alive (feat. Drake)
13 J. Cole  –  1 0 0 . m i l ‘ (with Bas)
14 jxdn  –  ANGELS & DEMONS
15 The White Stripes  –  Seven Nation Army
16 Nirvana  –  Come As You Are
17 Sublime  –  Santeria
18 100 gecs  –  Hollywood Baby
19 Lovejoy  –  Call Me What You Like


For each song that you get from current_user_saved_tracks, the variable will have two pieces ('added at', and 'track')

In [7]:
print('you have', results['total'], 'saved songs')

you have 4412 saved songs


In [9]:
profile

{'display_name': 'Dante Goss',
 'external_urls': {'spotify': 'https://open.spotify.com/user/1218158724'},
 'href': 'https://api.spotify.com/v1/users/1218158724',
 'id': '1218158724',
 'images': [{'url': 'https://scontent-atl3-1.xx.fbcdn.net/v/t39.30808-1/287739885_5170783559641650_7329131775805179735_n.jpg?stp=cp0_dst-jpg_p50x50&_nc_cat=108&ccb=1-7&_nc_sid=dbb9e7&_nc_ohc=MLQy0-zrAaoAX8LGpN5&_nc_ht=scontent-atl3-1.xx&edm=AP4hL3IEAAAA&oh=00_AfD1LdFLELVwMMa0UNWK-z-boyQ6Rn1PcvzJvFqojvExYw&oe=64E38975',
   'height': 64,
   'width': 64},
  {'url': 'https://scontent-atl3-1.xx.fbcdn.net/v/t39.30808-1/287739885_5170783559641650_7329131775805179735_n.jpg?stp=dst-jpg_p320x320&_nc_cat=108&ccb=1-7&_nc_sid=0c64ff&_nc_ohc=MLQy0-zrAaoAX8LGpN5&_nc_ht=scontent-atl3-1.xx&edm=AP4hL3IEAAAA&oh=00_AfAOx10JDaS5Z7iKHqKil7BfIJmWBRGp4g0CBFwNp6ji5A&oe=64E38975',
   'height': 300,
   'width': 300}],
 'type': 'user',
 'uri': 'spotify:user:1218158724',
 'followers': {'href': None, 'total': 15},
 'country': 'US',
 'p

In [10]:
# item is a dictionary
print('track id: ', item['track']['id'])

print('track name: ', item['track']['name'])

print('track length in seconds: ',item['track']['duration_ms']/1000)

print('album name: ',item['track']['album']['name'])

print('first artist: ', item['track']['artists'][0]['name'])

print('number of artists: ', len(item['track']['artists']))


track id:  2QF8FbGBTXTzm0CRUWqndE
track name:  Call Me What You Like
track length in seconds:  226.96
album name:  Wake Up & It's Over
first artist:  Lovejoy
number of artists:  1


In [45]:
# First get a loop that will capture the data we want into a df
d =[]
scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
result = sp.current_user_saved_tracks(limit = 20, offset = 0, market = None)
for item in enumerate(result['items']):
    d.append(
        {
            'Track_Name' : item['track']['name'],
            'Track_Id' : item['track']['id'],
            'Artist_Name' : item['track']['artists'][0]['name'],
            'Artist_Num' : len(item['track']['artists']),
            'Track_Length' : (item['track']['duration_ms']/1000)
        }
    )
d2 = pd.DataFrame(d)
d2.head()

# This loop didnt work

TypeError: tuple indices must be integers or slices, not str

In [ ]:
# Now lets make a new loop

# Initialize df
df = pd.DataFrame(columns=['Track_Name','Track_ID','Artist_Name','Artist_Num','Track_Len'])

# Set scope
scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"
# Authorize
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
# Get saved tracks
result = sp.current_user_saved_tracks(limit = 20, offset = 0, market = None)
# Loop through Saved tracks
for item in result['items']:
    track = item['track']
    df = df.append({
        'Track_Name': track['name'],
        'Track_ID' : track['id'],
        'Artist_Name' : track['artists'][0]['name'],
        'Artist_Num' : len(track['artists']),
        'Track_Len' : track['duration_ms']/1000
    })
# Note given that frame.append is deprecated and will be removed. Use pandas.concat

In [47]:
# Loop 3

# Initialize df
df = []
# Set scope
scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"
# Authorize
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
# Get saved tracks
result = sp.current_user_saved_tracks(limit = 20, offset = 0, market = None)

# Loop through saved tracks
for item in result['items']:
    track = item['track']
    
    # Assign Variables
    Track_Name = track['name']
    Track_ID = track['id']
    Artist_Name = track['artists'][0]['name']
    Artist_ID = track['artists'][0]['id']
    Artist_Num = len(track['artists'])
    Track_Len = track['duration_ms']/1000
    # Add values to df
    df.append({
        'Track_Name' : Track_Name,
        'Track_ID' : Track_ID,
        'Artist_Name' : Artist_Name,
        'Artist_ID' : Artist_ID,
        'Artist_Num' : Artist_Num,
        'Track_Len' : Track_Len
    })
df = pd.DataFrame(df, columns=['Track_Name','Track_ID','Artist_Name','Artist_ID','Artist_Num','Track_Len'])
print(df)
    

                                        Track_Name                Track_ID  \
0          Watch This - ARIZONATEARS Pluggnb Remix  0FA4wrjDJvJTTU8AepZTup   
1                                 Don't Understand  4MTuL20LF3pWebeJbcNh7p   
2                                         3 Nights  0uI7yAKUf52Cn7y3sYyjiX   
3             Double Negative (Skeleton Milkshake)  7ACT6YaXbYvl7hRWEOOEHQ   
4                                   Something Real  444vevlQjTnKioLLncteGv   
5                               My Own Worst Enemy  33iv3wnGMrrDugd7GBso1z   
6   h u n g e r . o n . h i l l s i d e (with Bas)  5BwQjRasNcdRPuVWKcHto2   
7                a p p l y i n g . p r e s s u r e  1d7q712nXjG98HiwHk7HFS   
8          m y . l i f e (with 21 Savage & Morray)  1D3z6HTiQsNmZxjl7F7eoG   
9              p u n c h i n ‘ . t h e . c l o c k  57ZUX6TNyKLBydAdVVd02x   
10                                 9 5 . s o u t h  5R691ipUYRDYW6ehapjoj6   
11                                       a m a r i  2cnKST6T9qUo